In [1]:
import pandas as pd
import mysql.connector

In [2]:
# Cargamos el archivo de ataques
df_ataques = pd.read_csv("../data/04-tiburon_4.csv", index_col = 0)
df_ataques.head(2)

,year,type,country,activity,age,species,month,fatal,gender
0,2018,Boating,usa,Paddling,57.0,White,Jun,N,F
1,2018,Unprovoked,brazil,Swimming,18.0,Tiger,Jun,Y,M


In [3]:
# Con los valores unicos para pais, creamos una lista para crear un nuevo dataframe
lista_paises = df_ataques.country.unique().tolist()

In [4]:
# Creamos un dataframe con los paises, asignandoles un id a cada uno
df_paises = pd.DataFrame(lista_paises).reset_index()
df_paises.rename(columns={0:"country", "index": "id_country"}, inplace=True)

In [5]:
df_paises.head(2)

,id_country,country
0,0,usa
1,1,brazil


In [6]:
# Guardamos la tabla paises por si la necesitamos en otro momento
df_paises.to_csv("../data/00-paises.csv")

In [7]:
df_ataques.shape

(1669, 9)

In [8]:
# Unimos el df de los ataques al de paises por la columna country, para crear la columna id_country
df_ataques = df_ataques.merge(df_paises, how= "inner", on= "country")
df_ataques.sample(2)

,year,type,country,activity,age,species,month,fatal,gender,id_country
443,1998,Unprovoked,usa,Walking,13.0,Unspecified,Nov,N,M,0
943,1993,Unprovoked,australia,Hookah diving,27.0,Tiger,Nov,Y,M,2


In [9]:
df_ataques.shape

(1669, 10)

In [10]:
# Cargamos el archivo del clima
df_climas = pd.read_csv("../data/00-datos_clima.csv", index_col= 0)
df_climas.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country,latitud,longitud
0,3,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 150, 'speed':...",16,6,4,1024,none,0,0,110,2,usa,39.78373,-100.445882
1,6,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 6}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 265, 'speed':...",15,6,6,1025,none,0,0,220,2,usa,39.78373,-100.445882


In [11]:
df_climas.shape

(320, 19)

In [12]:
# Unimos el df del clima al de pais para crear la columna id_country
df_climas = df_climas.merge(df_paises, how= "inner", on= "country")
df_climas.sample(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country,latitud,longitud,id_country
135,24,5,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 60, 'speed': ...",23,2,10,1020,none,0,0,65,3,south africa,-28.816624,24.991639,6
269,42,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 15}, {'layer': '900m...","[{'layer': '950mb', 'direction': 80, 'speed': ...",26,-1,12,1011,none,1,0,75,3,papua new guinea,-5.681607,144.248908,21


# Pair Programming ETL Carga I

Es el momento de meter todos nuestros datos en SQL 💪🏽!!! En este ejercicio nos crearemos dos tablas en una BBDD creada por nosotras. Una de las tablas contendrá la información que obtuvimos de los ejercicios de pair programming de Limpieza, es decir, el data set de ataques de tiburones limpito. La segunda tabla tendrá la información obtenida en el ejercicio de pair de ETL 1.

📌 Nota Todo lo tendremos que hacer desde jupyter notebook

1. Cread la BBDD con el nombre de tiburones.

2. Cread las tablas de la BBDD:
    - Tabla ataques
    - Tabla clima

3. BONUS Insertar los datos en las tablas.
📌 Nota Esta parte del pair es optativa y no será considerada para la evaluación

🚨 En caso de que no tengáis los datos unidos de la sesión anterior, tenéis un csv datos_union_clima_ataques con todos los datos que necesitareis para este ejercicio

Creamos una base de datos que se llame Tiburones

In [13]:
# Creamos la función para crear la base de datos
def crear_bbdd(nombre_bbdd):

    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password='AlumnaAdalab' 
    )
    print("Conexión realizada con éxito")
    
    mycursor = mydb.cursor()

    try:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
        print(mycursor)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [14]:
mydb = crear_bbdd("tiburones")

Conexión realizada con éxito
MySQLCursor: CREATE DATABASE IF NOT EXISTS tiburones;


In [15]:
# Creamos la función para crear las tablas
def crear_tabla(nombre_bbdd, contraseña, query):
    
    # nos conectamsos con el servidor usando el conector de sql
    cnx = mysql.connector.connect(user='root', password=f"{contraseña}",
                                     host='127.0.0.1', database=f"{nombre_bbdd}")
    # iniciamos el cursor
    mycursor = cnx.cursor()
    
    # intentamos hacer la query
    try: 
        mycursor.execute(query)
        cnx.commit() 
    # en caso de que podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [16]:
# Creamos una función para insertar los datos
def insercion_datos(nombrebbdd, contraseña, query):
    cnx = mysql.connector.connect(user='root', password=f"{contraseña}",
                                host='127.0.0.1', database=f"{nombrebbdd}")
    mycursor = cnx.cursor()
    
    try: 
        # Le añadimos un escape para poder subir los datos a pesar de tener una foreign key
        mycursor.execute("SET foreign_key_checks = 0;")
        mycursor.execute(query)
        cnx.commit()
        mycursor.execute("SET foreign_key_checks = 1;")

    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [17]:
# Query para crear la tabla paises
tabla_paises = '''
CREATE TABLE IF NOT EXISTS `tiburones`.`paises` (
  `country` VARCHAR (45) NOT NULL,
  `id_country` INT NOT NULL,
  PRIMARY KEY (`id_country`))
  ;'''

In [18]:
# Hacemos el info para ir viendo los nombres y tipo de datos de las columnas
df_climas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320 entries, 0 to 319
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   timepoint          320 non-null    int64  
 1   cloudcover         320 non-null    int64  
 2   highcloud          320 non-null    int64  
 3   midcloud           320 non-null    int64  
 4   lowcloud           320 non-null    int64  
 5   rh_profile         320 non-null    object 
 6   wind_profile       320 non-null    object 
 7   temp2m             320 non-null    int64  
 8   lifted_index       320 non-null    int64  
 9   rh2m               320 non-null    int64  
 10  msl_pressure       320 non-null    int64  
 11  prec_type          320 non-null    object 
 12  prec_amount        320 non-null    int64  
 13  snow_depth         320 non-null    int64  
 14  wind10m.direction  320 non-null    int64  
 15  wind10m.speed      320 non-null    int64  
 16  country            320 non

In [19]:
# Creamos la query para crear la tabla clima, con una foreign key a paises
tabla_clima = '''
CREATE TABLE IF NOT EXISTS `tiburones`.`clima` (
  `id_clima` INT NOT NULL AUTO_INCREMENT,
  `timepoint` INT NOT NULL,
  `cloudcover` INT NOT NULL,
  `highcloud` INT NOT NULL,
  `midcloud` INT NOT NULL,
  `lowcloud` INT NOT NULL,
  `rh_profile` VARCHAR (45) NOT NULL,
  `wind_profile` VARCHAR (45) NOT NULL,
  `temp2m` INT NOT NULL,
  `lifted_index` INT NOT NULL,
  `rh2m` INT NOT NULL,
  `msl_pressure` INT NOT NULL,
  `prec_type` VARCHAR (45) NOT NULL,
  `prec_amount` INT NOT NULL,
  `snow_depth` INT NOT NULL,
  `wind10m.direction` INT NOT NULL,
  `wind10m.speed` INT NOT NULL,
  `country` VARCHAR (45) NOT NULL,
  `latitud` DECIMAL NOT NULL,
  `longitud` DECIMAL NOT NULL,
  `id_country` INT NOT NULL,
  PRIMARY KEY (`id_clima`),
  INDEX `fk_ataques_paises1_idx` (`id_country` ASC) ,
  CONSTRAINT `fk_clima_paises` 
    FOREIGN KEY (`id_country`) 
    REFERENCES `paises`(`id_country`))
  ENGINE = InnoDB
  ;'''

In [20]:
# Hacemos un info para ir viendo los tipos de datos y los nombres de las columnas
df_ataques.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1669 entries, 0 to 1668
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        1669 non-null   int64  
 1   type        1669 non-null   object 
 2   country     1669 non-null   object 
 3   activity    1669 non-null   object 
 4   age         1669 non-null   float64
 5   species     1669 non-null   object 
 6   month       1669 non-null   object 
 7   fatal       1669 non-null   object 
 8   gender      1669 non-null   object 
 9   id_country  1669 non-null   int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 143.4+ KB


In [21]:
# Creamos la query para crear la tabla ataques, con una foreign key a paises
tabla_ataques = """
CREATE TABLE IF NOT EXISTS `ataques` (
    `id_ataques` INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    `year` INT NOT NULL,
    `type` VARCHAR(200) NOT NULL,
    `country` VARCHAR(45) NOT NULL,
    `activity` VARCHAR(200) NOT NULL,
    `age` INT NOT NULL,
    `species` VARCHAR(200) NOT NULL,
    `month` VARCHAR(200) NOT NULL,
    `fatal` VARCHAR(200) NOT NULL,
    `gender` VARCHAR(200) NOT NULL,
    `id_country` INT NOT NULL,
    INDEX `fk_ataques_paises1_idx` (`id_country` ASC) ,
    CONSTRAINT `fk_ataques_paises` 
        FOREIGN KEY (`id_country`) 
        REFERENCES `paises`(`id_country`))
    ENGINE = InnoDB
    ;"""

Llamamos a la función para crear las tablas

In [22]:
crear_tabla("tiburones", "AlumnaAdalab", tabla_paises)

In [23]:
crear_tabla("tiburones", "AlumnaAdalab", tabla_clima)

In [24]:
# Llamamos a la función para crear las dos tablas en nuestra BBD
crear_tabla("tiburones", "AlumnaAdalab", tabla_ataques)

Llamamos a la función para insertar los datos de la tabla ataques como prueba

In [25]:
# Al intentar cargar los datos, vemos que tenemos un error por una columna con una comilla de más. Lo reemplazamos por un espacio vacío
df_ataques["activity"] = df_ataques["activity"].replace(r'"', '', regex= True)

In [26]:
for indice, fila in df_ataques.iterrows(): # iteramos por el dataframe.
    
    # definimos nuestra query, igual que si lo hicieramos en workbench. ⚠️ Como estamos definiendo nuestra query en varias líneas usamos las triples comillas
    # lo valores que introduciremos serán los del dataframe que estamos iterando, por lo que usaremos los formats de los strings. 
    
    query_ataques = f"""
            INSERT INTO ataques (year, type, country, activity, age, species, month, fatal, gender, id_country)
            VALUES ( "{fila['year']}", "{fila['type']}", "{fila['country']}", "{fila['activity']}", "{fila['age']}", "{fila['species']}", "{fila['month']}", "{fila['fatal']}", "{fila['gender']}", "{fila['id_country']}");
            """
    # una vez definida la query llamamos a la función que nos inserta los datos. 
    insercion_datos("tiburones", "AlumnaAdalab", query_ataques)

In [27]:
# Guardamos por si los necesitamos proximamente
df_ataques.to_csv("../data/06-tiburon_4.csv")
df_climas.to_csv("../data/01-datos_clima.csv")